In [2]:
from master_bert import MASTERModel
import pickle
import numpy as np
import time

from utils import load_all_csv_data_with_market_indexes, load_all_csv_data_without_index, csvs_to_qlib_df, PandasDataLoader
# Please install qlib first before load the data.

# Qlib
# import qlib
# from qlib.config import REG_US           # S&P 500 is a US market
# qlib.init(provider_uri=".", region=REG_US)   # provider_uri just needs to exist





# ------------------------------------------------------------
# 1.  Init Qlib and build *one* handler
import qlib, pandas as pd, numpy as np, torch
qlib.init()                               # client mode is fine

from qlib.data.dataset.loader import StaticDataLoader
from qlib.data.dataset.handler import DataHandlerLP
from qlib.data.dataset import TSDatasetH          # <-- here
from qlib.data.dataset.processor import (
    DropnaProcessor, CSZScoreNorm, DropnaLabel,
)

# your tensor, names, dates exactly as before  ----------------
stock_tensor, stock_names, feature_names = load_all_csv_data_without_index()
# stock_tensor, stock_names, feature_names = load_all_csv_data_with_market_indexes()
N, T, K   = stock_tensor.shape
print("Shape: ", stock_tensor.shape)
# dates     = pd.read_csv("data/enriched/market_indexes_aggregated.csv")["Date"]
# dates = pd.to_datetime(                     # <-- NEW
#     pd.read_csv("data/enriched/market_indexes_aggregated.csv")["Date"]
# )

dates = pd.to_datetime(                     # <-- NEW
    pd.read_csv("data/normalized/market_indexes_aggregated_normalized.csv")["Date"]
)

# tensor ➜ tidy multi-index frame --------------------------------
def tensor_to_df(tensor, inst, feats, dt_index):
    flat = tensor.numpy().reshape(N * T, K)
    idx  = pd.MultiIndex.from_product([dt_index, inst],
                                      names=["datetime", "instrument"])
    cols = pd.MultiIndex.from_product([["feature"], feats])
    return pd.DataFrame(flat, index=idx, columns=cols)

df_raw = tensor_to_df(stock_tensor, stock_names, feature_names, dates)

# # OLD: build a forward-return label
# df_raw[("label", "FWD_RET")] = (
#     df_raw[("feature", "Adjusted Close")]
#       .groupby("instrument").shift(-1) / df_raw[("feature", "Adjusted Close")] - 1
# )

# last_date = dates.iloc[-1]
# df_raw = df_raw.drop(index=last_date, level="datetime")


# MASTER uses a d-day rank-normalized return, which reflects each stock's relative performance within the market at a specific date
# Steps: 
# Look Ahead:
# For each stock, MASTER looks a few days into the future (like 5 days) to see how much the price goes up or down.

# Calculate Return:
# It calculates the percentage change in price over those days — this is the raw return.

# Compare Stocks:
# On each day, it compares the returns of all stocks to see which ones performed better or worse.

# Z-score Normalization:
# It transforms those returns into standard scores (z-scores), so you know how each stock ranks relative to the others that day.

# Final Label:
# The model learns to predict this ranked performance score, not just the raw return.

# Oss: “The lookback window length T and prediction interval d are set as 8 and 5 respectively.” -- MaSTER paper

# Step 1: Compute d-day forward return
d = 5  # prediction interval
df_raw[("label", "FWD_RET")] = (
    df_raw[("feature", "Adjusted Close")]
      .groupby("instrument")
      .shift(-d) / df_raw[("feature", "Adjusted Close")] - 1
)

# Drop the last d rows since they can't have valid forward returns
for i in range(d):
    df_raw = df_raw.drop(index=dates.iloc[-(i+1)], level="datetime")

# Step 2: Z-score normalization across stocks (per date)
df_raw[("label", "Z_RET")] = (
    df_raw[("label", "FWD_RET")]
    .groupby("datetime")
    .transform(lambda x: (x - x.mean()) / x.std())
)

df_raw = df_raw.drop(columns=[("label", "FWD_RET")])

# handler with learn / infer processors ------------------------
proc_feat = [
    {"class": "DropnaProcessor", "kwargs": {"fields_group": "feature"}},
    # {"class": "CSZScoreNorm",   "kwargs": {"fields_group": "feature"}}, # slows down debugging
]

# proc_feat = [
#     {"class": "CSZScoreNorm",   "kwargs": {"fields_group": "feature"}},
# ]

# proc_feat = [
#     {"class": "Fillna",          # <— correct name
#      "kwargs": {"fields_group": "feature", "fill_value": 0}},  # zero-fill; choose ffill/bfill/etc. if you like
#     {"class": "CSZScoreNorm",
#      "kwargs": {"fields_group": "feature"}},
# ]



[2991408:MainThread](2025-05-31 15:02:42,886) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[2991408:MainThread](2025-05-31 15:02:42,887) WARNING - qlib.Initialization - [__init__.py:64] - auto_path is False, please make sure None is mounted
[2991408:MainThread](2025-05-31 15:02:43,334) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[2991408:MainThread](2025-05-31 15:02:43,334) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/gabrielecarrino/.qlib/qlib_data/cn_data')}


Shape:  torch.Size([336, 3764, 227])


In [3]:
# Convert MultiIndex DataFrame to standard DataFrame
df_standard = df_raw.reset_index()

# Display the first 5 rows
print(df_standard.head())


    datetime instrument   feature                                          \
                              Low      Open    Volume      High     Close   
0 2008-01-02          A -0.754933 -0.751857 -0.368570 -0.759748 -0.762322   
1 2008-01-02        AAL -0.761569 -0.761598 -0.283911 -0.771496 -0.771803   
2 2008-01-02        AAP -0.792098 -0.780552  0.669788 -0.784548 -0.802615   
3 2008-01-02       AAPL -0.783337 -0.786870  0.652856 -0.783765 -0.787604   
4 2008-01-02        ABC -0.780683 -0.786344  0.998749 -0.787159 -0.788921   

                                            ...                                \
  Adjusted Close ABER_ZG_5_15 ABER_SG_5_15  ...   VWMA_10       WCP  WILLR_14   
0      -0.764863    -0.742809    -0.746216  ... -0.754006 -0.759885 -0.528163   
1      -0.773410    -0.749188    -0.753052  ... -0.751685 -0.769232 -0.656546   
2      -0.801190    -0.760590    -0.763502  ... -0.757157 -0.795497 -1.848760   
3      -0.787656    -0.770829    -0.773050  ... -0.7618

In [4]:
# Get unique sorted dates
unique_dates = df_raw.index.get_level_values("datetime").unique()
n_total = len(unique_dates)

# Compute split indices
n_train = int(0.8 * n_total)
n_valid = int(0.1 * n_total)
n_test  = n_total - n_train - n_valid  # remaining

# Slice date ranges
train_dates = unique_dates[:n_train]
valid_dates = unique_dates[n_train:n_train + n_valid]
test_dates  = unique_dates[n_train + n_valid:]

# Create splits
df_train = df_raw.loc[train_dates]
df_valid = df_raw.loc[valid_dates]
df_test  = df_raw.loc[test_dates]

# Show a quick check
print("Train:", df_train.index.get_level_values('datetime').min(), "->", df_train.index.get_level_values('datetime').max())
print("Valid:", df_valid.index.get_level_values('datetime').min(), "->", df_valid.index.get_level_values('datetime').max())
print("Test: ", df_test.index.get_level_values('datetime').min(), "->", df_test.index.get_level_values('datetime').max())


Train: 2008-01-02 00:00:00 -> 2019-12-10 00:00:00
Valid: 2019-12-11 00:00:00 -> 2021-06-08 00:00:00
Test:  2021-06-09 00:00:00 -> 2022-12-05 00:00:00


## XGBOOST

In [5]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Flatten DataFrames
train_df = df_train.reset_index()
valid_df = df_valid.reset_index()
test_df  = df_test.reset_index()

# Separate features and target
FEATURE_COLS = [col[1] for col in df_raw.columns if col[0] == "feature"]
TARGET_COL = "Z_RET"

# Extract feature matrices and target vectors
X_train = train_df[[("feature", f) for f in FEATURE_COLS]].values
y_train = train_df[("label", TARGET_COL)].values

X_valid = valid_df[[("feature", f) for f in FEATURE_COLS]].values
y_valid = valid_df[("label", TARGET_COL)].values

X_test = test_df[[("feature", f) for f in FEATURE_COLS]].values
y_test = test_df[("label", TARGET_COL)].values

# Optional: Standard scaling (depending on if you did CSZScoreNorm)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_valid = scaler.transform(X_valid)
# X_test  = scaler.transform(X_test)

# Train XGBoost regressor
model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=False,
)

# Evaluate
y_pred_test = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_test)
print(f"Test RMSE: {rmse:.4f}")


Test RMSE: 0.8357


In [ ]:
def calc_ic(pred, label):
    df = pd.DataFrame({'pred': pred, 'label': label})
    ic = df['pred'].corr(df['label'])
    ric = df['pred'].corr(df['label'], method='spearman')
    return ic, ric

def zscore(x):
    return (x - x.mean()) / (x.std() + 1e-8)

# Build Series with correct multi-index (datetime, instrument)
test_index = df_test.index
pred_series = pd.Series(y_pred_test, index=test_index)
label_series = pd.Series(y_test, index=test_index)

# Compute daily IC and RIC
ics, rics = [], []
for dt, pred_slice in pred_series.groupby(level="datetime"):
    label_slice = label_series.loc[dt]

    # FIX: align values only, discard index
    ic, ric = calc_ic(pred_slice.values, label_slice.values)
    ics.append(ic)
    rics.append(ric)

# Portfolio vs benchmark returns
daily_port_ret, daily_bench_ret = [], []

for dt, pred_slice in pred_series.groupby(level="datetime"):
    label_slice = label_series.loc[dt]

    # Portfolio: top 30 predicted
    top30_idx = pred_slice.nlargest(30).index
    port_ret = label_series.loc[top30_idx].mean()
    daily_port_ret.append(port_ret)

    # Benchmark: equal-weight average
    bench_ret = label_slice.mean()
    daily_bench_ret.append(bench_ret)

# Compute AR (active return) and IR (information ratio)
daily_port_ret = np.array(daily_port_ret)
daily_bench_ret = np.array(daily_bench_ret)
active_ret = daily_port_ret - daily_bench_ret

AR = active_ret.mean()
IR = AR / (active_ret.std() + 1e-12)

# Final metrics
metrics = {
    "IC":     np.mean(ics),
    "ICIR":   np.mean(ics) / (np.std(ics) + 1e-12),
    "RIC":    np.mean(rics),
    "RICIR":  np.mean(rics) / (np.std(rics) + 1e-12),
    "AR":     AR,
    "IR":     IR,
}

# Print metrics
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


# Transformer

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error
import numpy as np

# ──────────────────────────────────────────────────────────────
# 1.  Prepare tensors and loaders
# ──────────────────────────────────────────────────────────────
BATCH_SIZE   = 512
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
N_FEATURES   = X_train.shape[1]

def make_loader(X, y, shuffle):
    X_t = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # (batch, 1, features)
    y_t = torch.tensor(y, dtype=torch.float32)
    ds  = TensorDataset(X_t, y_t)
    return DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle, drop_last=False)

train_loader = make_loader(X_train, y_train, shuffle=True)
valid_loader = make_loader(X_valid, y_valid, shuffle=False)
test_loader  = make_loader(X_test,  y_test,  shuffle=False)

# ──────────────────────────────────────────────────────────────
# 2.  Define a very small Transformer
# ──────────────────────────────────────────────────────────────
class TransformerRegressor(nn.Module):
    def __init__(self, n_features, d_model=128, nhead=4, num_layers=1, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(n_features, d_model)
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=4 * d_model,
            dropout=dropout,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.head    = nn.Linear(d_model, 1)

    def forward(self, x):                     # x: (batch, 1, n_features)
        x = self.input_proj(x)                # -> (batch, 1, d_model)
        x = self.encoder(x)                   # -> (batch, 1, d_model)
        x = self.head(x[:, 0, :])             # first (and only) token
        return x.squeeze(-1)                  # -> (batch,)

model = TransformerRegressor(N_FEATURES).to(DEVICE)

# ──────────────────────────────────────────────────────────────
# 3.  Train
# ──────────────────────────────────────────────────────────────
criterion   = nn.MSELoss()
optimizer   = torch.optim.Adam(model.parameters(), lr=1e-3)
patience    = 10
best_rmse   = np.inf
wait_epochs = 0
EPOCHS      = 5

for epoch in range(1, EPOCHS + 1):
    # ---- training ----
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

    # ---- validation ----
    model.eval()
    with torch.no_grad():
        preds_v, labels_v = [], []
        for xb, yb in valid_loader:
            xb = xb.to(DEVICE)
            preds_v.append(model(xb).cpu())
            labels_v.append(yb)
        preds_v  = torch.cat(preds_v).numpy()
        labels_v = torch.cat(labels_v).numpy()
        rmse_v   = mean_squared_error(labels_v, preds_v)

    if rmse_v < best_rmse:
        best_rmse   = rmse_v
        wait_epochs = 0
        best_state  = model.state_dict()
    else:
        wait_epochs += 1
        if wait_epochs >= patience:
            print(f"Early stop at epoch {epoch:3d}  |  best val-RMSE = {best_rmse:.4f}")
            break

# load the best model weights
model.load_state_dict(best_state)

# ──────────────────────────────────────────────────────────────
# 4.  Test set inference
# ──────────────────────────────────────────────────────────────
model.eval()
with torch.no_grad():
    preds_test = []
    for xb, _ in test_loader:
        xb = xb.to(DEVICE)
        preds_test.append(model(xb).cpu())
    y_pred_test = torch.cat(preds_test).numpy()

rmse = mean_squared_error(y_test, y_pred_test)
print(f"Test RMSE: {rmse:.4f}")


Test RMSE: 0.9315


In [ ]:
def calc_ic(pred, label):
    df = pd.DataFrame({'pred': pred, 'label': label})
    ic = df['pred'].corr(df['label'])
    ric = df['pred'].corr(df['label'], method='spearman')
    return ic, ric

def zscore(x):
    return (x - x.mean()) / (x.std() + 1e-8)

# Build Series with correct multi-index (datetime, instrument)
test_index = df_test.index
pred_series = pd.Series(y_pred_test, index=test_index)
label_series = pd.Series(y_test, index=test_index)

# Compute daily IC and RIC
ics, rics = [], []
for dt, pred_slice in pred_series.groupby(level="datetime"):
    label_slice = label_series.loc[dt]

    # FIX: align values only, discard index
    ic, ric = calc_ic(pred_slice.values, label_slice.values)
    ics.append(ic)
    rics.append(ric)

# Portfolio vs benchmark returns
daily_port_ret, daily_bench_ret = [], []

for dt, pred_slice in pred_series.groupby(level="datetime"):
    label_slice = label_series.loc[dt]

    # Portfolio: top 30 predicted
    top30_idx = pred_slice.nlargest(30).index
    port_ret = label_series.loc[top30_idx].mean()
    daily_port_ret.append(port_ret)

    # Benchmark: equal-weight average
    bench_ret = label_slice.mean()
    daily_bench_ret.append(bench_ret)

# Compute AR (active return) and IR (information ratio)
daily_port_ret = np.array(daily_port_ret)
daily_bench_ret = np.array(daily_bench_ret)
active_ret = daily_port_ret - daily_bench_ret

AR = active_ret.mean()
IR = AR / (active_ret.std() + 1e-12)

# Final metrics
metrics = {
    "IC":     np.mean(ics),
    "ICIR":   np.mean(ics) / (np.std(ics) + 1e-12),
    "RIC":    np.mean(rics),
    "RICIR":  np.mean(rics) / (np.std(rics) + 1e-12),
    "AR":     AR,
    "IR":     IR,
}

# Print metrics
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


# Cross-validation on Transformer

In [18]:
def train_trans():


    # ──────────────────────────────────────────────────────────────
    # 1.  Prepare tensors and loaders
    # ──────────────────────────────────────────────────────────────
    BATCH_SIZE   = 512
    DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
    N_FEATURES   = X_train.shape[1]

    def make_loader(X, y, shuffle):
        X_t = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # (batch, 1, features)
        y_t = torch.tensor(y, dtype=torch.float32)
        ds  = TensorDataset(X_t, y_t)
        return DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle, drop_last=False)

    train_loader = make_loader(X_train, y_train, shuffle=True)
    valid_loader = make_loader(X_valid, y_valid, shuffle=False)
    test_loader  = make_loader(X_test,  y_test,  shuffle=False)

    # ──────────────────────────────────────────────────────────────
    # 2.  Define a very small Transformer
    # ──────────────────────────────────────────────────────────────
    class TransformerRegressor(nn.Module):
        def __init__(self, n_features, d_model=128, nhead=4, num_layers=1, dropout=0.1):
            super().__init__()
            self.input_proj = nn.Linear(n_features, d_model)
            enc_layer = nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=nhead,
                dim_feedforward=4 * d_model,
                dropout=dropout,
                batch_first=True,
            )
            self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
            self.head    = nn.Linear(d_model, 1)

        def forward(self, x):                     # x: (batch, 1, n_features)
            x = self.input_proj(x)                # -> (batch, 1, d_model)
            x = self.encoder(x)                   # -> (batch, 1, d_model)
            x = self.head(x[:, 0, :])             # first (and only) token
            return x.squeeze(-1)                  # -> (batch,)

    model = TransformerRegressor(N_FEATURES).to(DEVICE)

    # ──────────────────────────────────────────────────────────────
    # 3.  Train
    # ──────────────────────────────────────────────────────────────
    criterion   = nn.MSELoss()
    optimizer   = torch.optim.Adam(model.parameters(), lr=1e-3)
    patience    = 10
    best_rmse   = np.inf
    wait_epochs = 0
    EPOCHS      = 5

    for epoch in range(1, EPOCHS + 1):
        # ---- training ----
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

        # ---- validation ----
        model.eval()
        with torch.no_grad():
            preds_v, labels_v = [], []
            for xb, yb in valid_loader:
                xb = xb.to(DEVICE)
                preds_v.append(model(xb).cpu())
                labels_v.append(yb)
            preds_v  = torch.cat(preds_v).numpy()
            labels_v = torch.cat(labels_v).numpy()
            rmse_v   = mean_squared_error(labels_v, preds_v)

        if rmse_v < best_rmse:
            best_rmse   = rmse_v
            wait_epochs = 0
            best_state  = model.state_dict()
        else:
            wait_epochs += 1
            if wait_epochs >= patience:
                print(f"Early stop at epoch {epoch:3d}  |  best val-RMSE = {best_rmse:.4f}")
                break

    # load the best model weights
    model.load_state_dict(best_state)

    # ──────────────────────────────────────────────────────────────
    # 4.  Test set inference
    # ──────────────────────────────────────────────────────────────
    model.eval()
    with torch.no_grad():
        preds_test = []
        for xb, _ in test_loader:
            xb = xb.to(DEVICE)
            preds_test.append(model(xb).cpu())
        y_pred_test = torch.cat(preds_test).numpy()

    rmse = mean_squared_error(y_test, y_pred_test)
    print(f"Test RMSE: {rmse:.4f}")

    def calc_ic(pred, label):
        df = pd.DataFrame({'pred': pred, 'label': label})
        ic = df['pred'].corr(df['label'])
        ric = df['pred'].corr(df['label'], method='spearman')
        return ic, ric

    def zscore(x):
        return (x - x.mean()) / (x.std() + 1e-8)

    # Build Series with correct multi-index (datetime, instrument)
    test_index = df_test.index
    pred_series = pd.Series(y_pred_test, index=test_index)
    label_series = pd.Series(y_test, index=test_index)

    # Compute daily IC and RIC
    ics, rics = [], []
    for dt, pred_slice in pred_series.groupby(level="datetime"):
        label_slice = label_series.loc[dt]

        # FIX: align values only, discard index
        ic, ric = calc_ic(pred_slice.values, label_slice.values)
        ics.append(ic)
        rics.append(ric)

    # Portfolio vs benchmark returns
    daily_port_ret, daily_bench_ret = [], []

    for dt, pred_slice in pred_series.groupby(level="datetime"):
        label_slice = label_series.loc[dt]

        # Portfolio: top 30 predicted
        top30_idx = pred_slice.nlargest(30).index
        port_ret = label_series.loc[top30_idx].mean()
        daily_port_ret.append(port_ret)

        # Benchmark: equal-weight average
        bench_ret = label_slice.mean()
        daily_bench_ret.append(bench_ret)

    # Compute AR (active return) and IR (information ratio)
    daily_port_ret = np.array(daily_port_ret)
    daily_bench_ret = np.array(daily_bench_ret)
    active_ret = daily_port_ret - daily_bench_ret

    AR = active_ret.mean()
    IR = AR / (active_ret.std() + 1e-12)

    # Final metrics
    metrics = {
        "IC":     np.mean(ics),
        "ICIR":   np.mean(ics) / (np.std(ics) + 1e-12),
        "RIC":    np.mean(rics),
        "RICIR":  np.mean(rics) / (np.std(rics) + 1e-12),
        "AR":     AR,
        "IR":     IR,
    }

    # Print metrics
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")
        
    return metrics




In [ ]:
ic = []
icir = []
ric = []
ricir = []


# New metrics
ar = []
ir = []
for seed in [i for i in range(5)]: 
    metrics = train_trans()

    ic.append(metrics['IC'])
    icir.append(metrics['ICIR'])
    ric.append(metrics['RIC'])
    ricir.append(metrics['RICIR'])
    ar.append(metrics['AR'])
    ir.append(metrics['IR'])

In [ ]:
from scipy.stats import t

# Sample size
n = len(ic)

# Degrees of freedom
df = n - 1

# t critical value for 95% confidence
t_crit = t.ppf(0.975, df)  # two-tailed

# Function to compute mean and 95% CI error
def ci_95(arr):
    mean = np.mean(arr)
    se = np.std(arr, ddof=1) / np.sqrt(len(arr))
    margin = t_crit * se
    return mean, margin

# Print each metric with 95% CI
print("IC: {:.4f} pm {:.4f}".format(*ci_95(ic)))
print("ICIR: {:.4f} pm {:.4f}".format(*ci_95(icir)))
print("RIC: {:.4f} pm {:.4f}".format(*ci_95(ric)))
print("RICIR: {:.4f} pm {:.4f}".format(*ci_95(ricir)))
print("AR: {:.4f} pm {:.4f}".format(*ci_95(ar)))
print("IR: {:.4f} pm {:.4f}".format(*ci_95(ir)))